In [2]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff # for dendogram
import numpy as np


In [8]:
path= r"data\src\esrs"
df0 = pl.read_excel(path+'\esrs.xlsx')
df10 = pl.read_excel(path+'\\report.xlsx',sheet_name='report',read_options={"header_row": 2})


In [9]:
df10

Directives,Directive Materiality,SubDirectives,SubDirective Materiality,Paragraph,Paragraph Materiality,Text,Type,Ecoscuse,Relevancy Score,__UNNAMED__10,__UNNAMED__11,__UNNAMED__12,__UNNAMED__13
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""E1""","""normal""","""E1.6""","""normal""","""AR 45e""","""normal""","""Aufschlüsselung der Scope-1-Em…",""" b) aus Heizöl (global): 737,2…",""" c) aus Dienstwagenflotte (GE…",""" d) aus Pellets (GER): 7,0 t C…",""" b) aus Fernwärmebezug (global…","""Measurement""","""❌""","""0.6127781867980957"""
"""S3""","""normal""","""S3.5""","""normal""","""AR 44 c""","""normal""","""die ESRS und die Corporate Sus…","""Target""","""❌""","""0.5771147608757019""",null,null,null,null
"""E2""","""normal""","""E2.4""","""normal""","""AR 25c""","""normal""","""A U F E I N E N B L I C K …","""Target""","""❌""","""0.5557408332824707""",null,null,null,null
"""E5""","""normal""","""E5.3""","""normal""","""23""","""normal""","""Zudem tauschen wir uns regelmä…","""Target""","""❌""","""0.5472809672355652""",null,null,null,null
"""E1""","""normal""","""E1.3""","""normal""","""29cii,16c""","""normal""","""Für 2024 planen wir eine Aktua…","""Target""","""❌""","""0.5471444725990295""",null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""E5""","""normal""","""E5.1""","""normal""","""15a""","""normal""","""Auch in Bezug auf Akquisitione…","""Target""","""❌""","""0.42076870799064636""",null,null,null,null
"""S1""","""normal""","""S1.6""","""normal""","""50a""","""normal""","""Über die Ergebnisse im Geschäf…","""Target""","""❌""","""0.42066067457199097""",null,null,null,null
"""E5""","""normal""","""E5.4""","""normal""","""31c""","""normal""","""Im aktuellen Geschäftsjahr sti…","""Measurement""","""❌""","""0.41953060030937195""",null,null,null,null


In [10]:
df11 = df10['SubDirectives','Directives'].with_columns(
                pl.when(pl.col("Directives").str.slice(0,length=1)=='E').then(pl.lit('Umwelt'))
                .when(pl.col("Directives").str.slice(0,length=1)=='S').then(pl.lit('Soziales'))
                .when(pl.col("Directives").str.slice(0,length=1)=='G').then(pl.lit('Governance'))
                .otherwise(pl.lit('Übergreifend')).alias("Parent")
                )
df11

SubDirectives,Directives,Parent
str,str,str
"""E1.6""","""E1""","""Umwelt"""
"""S3.5""","""S3""","""Soziales"""
"""E2.4""","""E2""","""Umwelt"""
"""E5.3""","""E5""","""Umwelt"""
"""E1.3""","""E1""","""Umwelt"""
…,…,…
"""E5.1""","""E5""","""Umwelt"""
"""S1.6""","""S1""","""Soziales"""
"""E5.4""","""E5""","""Umwelt"""


In [11]:
fig = px.sunburst(df11.sort(by='SubDirectives'),
            path=['Parent', 'Directives', 'SubDirectives'],
            width=800,height=800,
            title="<b>Report Analysis - Sunburst Chart</b>")
fig.update_traces(sort=False, selector=dict(type='sunburst'), textinfo='label+percent root')
fig.update_layout(title_x=0.5, margin=dict(t=50,)) 
fig.show()

In [12]:
fig = px.treemap(df11, 
                 path=['Parent', 'Directives', 'SubDirectives'], 
                 title="<b>Report Analysis - Treemap</b>",
                 width=1300, height=550
                 )
fig.update_traces(textinfo='label+percent root')
fig.update_layout(title_x=0.5, margin=dict(t=50,b=20))
fig.show()

In [13]:
fig = px.icicle(df11.sort(by='SubDirectives'),
                path=['Parent', 'Directives', 'SubDirectives'], 
                title="<b>Report Analysis - Icicle Chart</b>",
                height=800, width=800)
fig.update_traces(sort=False, textinfo='label+percent root')
fig.update_layout(margin=dict(l=0, r=20, t=50, b=20, pad=0),title_x=0.5)
fig.show()

In [14]:
df12 = df11['SubDirectives'].value_counts().sort(by='SubDirectives').with_columns(
                pl.when(pl.col("SubDirectives").str.slice(0,length=1)=='E').then(pl.lit('Umwelt'))
                .when(pl.col("SubDirectives").str.slice(0,length=1)=='S').then(pl.lit('Soziales'))
                .when(pl.col("SubDirectives").str.slice(0,length=1)=='G').then(pl.lit('Governance'))
                .otherwise(pl.lit('Übergreifend')).alias("Parent")
                ).with_columns(
                    pl.col('SubDirectives').str.slice(0,length=2).alias('Directives')
                )
df12

SubDirectives,count,Parent,Directives
str,u32,str,str
"""E1.1""",2,"""Umwelt""","""E1"""
"""E1.2""",1,"""Umwelt""","""E1"""
"""E1.3""",2,"""Umwelt""","""E1"""
"""E1.4""",9,"""Umwelt""","""E1"""
"""E1.6""",5,"""Umwelt""","""E1"""
…,…,…,…
"""S3.1""",1,"""Soziales""","""S3"""
"""S3.5""",1,"""Soziales""","""S3"""
"""S4.1""",4,"""Soziales""","""S4"""


In [15]:
list_custom_order = ["Umwelt","Soziales","Governance"]

df12 = pl.concat([df12.filter(pl.col('Parent')==list_custom_order[0]),
                  df12.filter(pl.col('Parent')==list_custom_order[1]),
                  df12.filter(pl.col('Parent')==list_custom_order[2])])
df12

SubDirectives,count,Parent,Directives
str,u32,str,str
"""E1.1""",2,"""Umwelt""","""E1"""
"""E1.2""",1,"""Umwelt""","""E1"""
"""E1.3""",2,"""Umwelt""","""E1"""
"""E1.4""",9,"""Umwelt""","""E1"""
"""E1.6""",5,"""Umwelt""","""E1"""
…,…,…,…
"""S4.4""",2,"""Soziales""","""S4"""
"""G1.1""",2,"""Governance""","""G1"""
"""G1.2""",1,"""Governance""","""G1"""


In [16]:
fig = px.bar(df12, x='SubDirectives', y='count', color='Parent', title="<b>Report Analysis - Grouped Bar Chart</b>")
fig.update_layout(title_x=0.5, legend_title=None, margin=dict(t=50))
fig.show()

In [17]:
fig = px.scatter(df12, x='SubDirectives', y='Directives', size='count', color='Parent', title="<b>Report Analysis - Bubble Chart</b>")
fig.update_layout(title_x=0.5, legend_title=None,margin=dict(t=50))
fig.show()

In [18]:
from plotly.subplots import make_subplots
# Extract unique parent categories for subplots
unique_parents = pl.DataFrame({"Parent" : list_custom_order})['Parent']

# Create a subplot figure for bubble charts
fig = make_subplots(rows=len(unique_parents), cols=1, 
                       #subplot_titles=unique_parents,
                        shared_xaxes=False, vertical_spacing=0.05,
                        x_title='SubDirectives',
                        y_title='count',)

# Loop through each unique parent and create a bubble chart
for i, parent in enumerate(unique_parents):
    # Filter the data for the current parent
    parent_data = df12.filter(pl.col('Parent') == parent)
    
    # Add a bubble chart for the current parent
    fig.add_trace(go.Scatter(
        x=parent_data['SubDirectives'],
        y=parent_data['count'],
        mode='markers',
        marker=dict(size=parent_data['count'] * 10, sizemode='area', sizeref=2. * max(parent_data['count']) / 100),
        name=parent
    ), row=i+1, col=1)

# Update layout
fig.update_layout(title=dict(text="<b>Report Analysis - Bubble Chart</b>"),
                  height=200 * len(unique_parents),
                  showlegend=True, title_x=0.5,
                  margin=dict(t=50))

# Display the subplot
fig.show()

In [19]:
E = [17, 7, 7, 10, 9]
S = [26, 6, 6, 6]
G = [10]

In [20]:
dict_ESG = {}
for x in [{'E'+str(i+1) : list(range(1,j+1))} for i,j in enumerate(E)]:
    dict_ESG.update(x)
for y in [{'S'+str(i+1) : list(range(1,j+1))} for i,j in enumerate(S)]:
    dict_ESG.update(y)
for z in [{'G'+str(i+1) : list(range(1,j+1))} for i,j in enumerate(G)]:
    dict_ESG.update(z)

In [21]:
#z = np.random.randint(1, 5, size=(len(E+S+G), max(E+S+G)))/4
z = np.random.choice(np.arange(0, 5), p=[0.05, 0.25, 0.25, 0.25, 0.2], size=(len(E+S+G), max(E+S+G)))/4
for i in range(z.shape[0]):
    for j in range(z.shape[1]):
        if j >= (E+S+G)[i]:
            z[i][j]=0

fig = px.imshow(z, labels=dict(x="SubDirectives", y="Directives"),
                x=list(range(1,max(E+S+G)+1)),
                y=list(dict_ESG)[:len(E+S+G)],
                color_continuous_scale=[(0, "white"), (0.25, "red"),(0.5,"yellow"), (0.75, "green"), (1, "green")],
                text_auto=False,title='<b>Report Analysis</b>',)
                #height=500,width=800)
# move x axis to top
fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_layout(title_x=0.5)
# hide colorbar legend
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [22]:
E+S+G

[17, 7, 7, 10, 9, 26, 6, 6, 6, 10]

In [23]:
data = {
    "Class": ["E4.SBM-3","E4.IRO-1","E4-1", "E4-2","E4-3","E4-4","E4-5","E4-6"],
    "yes": [200/6, 900/16,1600/18,1400/20,400/16,900/10,1900/27,100/6],
    "no": [400/6, 700/16,200/18,600/20,1200/16,100/10,800/27,500/6]
}

# Create a Polars DataFrame
df = pl.DataFrame(data)

# Plotly stacked bar plot
fig = go.Figure()

# Add Class1 data
fig.add_trace(go.Bar(
    y=df["Class"].to_list(),
    x=df["yes"].to_list(),
    name='yes',
    orientation='h',
    marker=dict(color='#8cd47e')
))

# Add Class2 data
fig.add_trace(go.Bar(
    y=df["Class"].to_list(),
    x=df["no"].to_list(),
    name='no',
    orientation='h',
    marker=dict(color='#ff6961')
))

# Update layout for the plot
fig.update_layout(
    barmode='stack',
    title='<b>Verteilung der Datenpunkte - E4</b>',title_x=0.5,
    xaxis=dict(title='Vollständigkeit %'),
    yaxis=dict(title='Datenpunkte'),
    showlegend=False,
    width=600, height=300, margin=dict(t=40,b=50, r=50,l=50)
)
fig.update_yaxes(autorange="reversed")
# Add annotations
#fig.update_traces(texttemplate='%{x:.1f}%', textposition='inside')

# Show the figure
fig.show()